#### В этом ноутбуке я обучаю две версии модели "Helsinki-NLP/opus-mt-ru-en" на двух версиях датасетов

In [ ]:
import evaluate
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, pipeline
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType, PeftModel

In [ ]:
# Загружаем модель и токенизатор
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [3]:
# Читаем датафрейм из файла
data_frame = pd.read_csv("data/full_data_ver_2.csv").drop('Unnamed: 0', axis=1)

# Переводим датафрейм в датасет
dataset = Dataset.from_pandas(data_frame)

# Токенизируем
def preprocess(example):
    model_inputs = tokenizer(example["Russian"], max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["English"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Делим на train/test
split_dataset = dataset.train_test_split(test_size=0.1, seed=42)
raw_train = split_dataset['train'] # Чистые колонки Russian, English на трейн
raw_test = split_dataset['test']  # Чистые колонки Russian, English на тест

# Токенизируем
tokenized_train = raw_train.map(preprocess)
tokenized_test = raw_test.map(preprocess)

# Убираем в токенизированном датасете колонки со словами
tokenized_train = tokenized_train.remove_columns(["Russian", "English"])
tokenized_test = tokenized_test.remove_columns(["Russian", "English"])


Map:   0%|          | 0/1135 [00:00<?, ? examples/s]c:\Users\mrpec\OneDrive\Рабочий стол\ML_HSE\Проект\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 127/127 [00:00<00:00, 2080.20 examples/s]


In [ ]:
# Конфиг для LoRA
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
## В первой версии обучения я брал только "q_proj", "k_proj", думая, что это поможет модельке не переобучиться

# Получаем модель для LoRA
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
# Тренируем
training_args = Seq2SeqTrainingArguments(
    output_dir="lora-opus-ru-en", # Этой директории нет, она осталась в коллабе
    per_device_train_batch_size=8,
    num_train_epochs=5,
    learning_rate=5e-4,
    logging_dir="logs",
    logging_steps=10,
    save_total_limit=1,
    save_steps=100,
    fp16=False,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
    eval_dataset=tokenized_test
)

trainer.train()

# Тут отображены эпохи для второй версии модели, обучал в колабе

<ipython-input-11-a28b1b5d66be>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,2.563000
20,2.563200
30,2.314400
40,2.085300
50,2.245300
60,2.183600
70,1.943300
80,1.882800
90,1.767600
100,1.911600


TrainOutput(global_step=710, training_loss=1.3519981585757832, metrics={'train_runtime': 1475.4796, 'train_samples_per_second': 3.839, 'train_steps_per_second': 0.481, 'total_flos': 61218667560960.0, 'train_loss': 1.3519981585757832, 'epoch': 5.0})

In [ ]:
# Сохраняем файлы модели
model.save_pretrained("model_files")
tokenizer.save_pretrained("model_files")

# Названия для папок с файлами изменены на my_LoRA_ver1 и my_LoRA_ver2

In [ ]:
base_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
model = PeftModel.from_pretrained(base_model, "my_LoRA_ver2")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

translator = pipeline("translation", model=model, tokenizer=tokenizer)  
print(translator("Цыпленок в медовой глазури с авокадо гриль и шпинатом")[0]['translation_text'])

Device set to use cpu


Chicken in honey glaze with grilled avocado and spinach


In [ ]:
# Мне захотелось слить базовую модель и две мои LoRA в две версии моделей

# Данные для моделей
base_model_name = "Helsinki-NLP/opus-mt-ru-en"
lora_ver1 = "my_LoRA_ver1"
lora_ver2 = "my_LoRA_ver2"

# Загружаем базовую модель и токенизатор
base_model_1 = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)
base_model_2 = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Загружаем LoRA поверх базовой модели
model_ver1 = PeftModel.from_pretrained(base_model_1, lora_ver1)
model_ver2 = PeftModel.from_pretrained(base_model_2, lora_ver2)

# Объединяем веса и выгружаем LoRA
merged_model_ver1 = model_ver1.merge_and_unload()
merged_model_ver2 = model_ver2.merge_and_unload()

# Сохраняем слитую модель и токенизатор

merged_model_ver1.save_pretrained("my_model_ver1")
tokenizer.save_pretrained("my_model_ver1")

merged_model_ver2.save_pretrained("my_model_ver2")
tokenizer.save_pretrained("my_model_ver2")

# Все файлы неслитых моделей были перемещены в "unfinished_models_files"

In [ ]:
# Оценка моделей
def metrics(model, tokenizer, test_set):

    # Делаю из датасета датафрейм
    if type(test_set) != "datasets.arrow_dataset.Dataset":
        test_set = test_set.to_pandas()
    else:
        pass

    # Грузим метрики
    bleu = evaluate.load("bleu")
    rouge = evaluate.load("rouge")
    chrf = evaluate.load("chrf")

    # Считаю предсказания
    predictions = []
    references = []

    for i, row in test_set.iterrows():
        inputs = tokenizer(row["Russian"], return_tensors="pt", truncation=True, padding=True)
        outputs = model.generate(**inputs, max_length=128)
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(pred)
        references.append([row["English"]])  

    # Считаю метрики
    bleu_ = bleu.compute(predictions=predictions, references=references)['bleu']
    chrf_ = chrf.compute(predictions=predictions, references=[r[0] for r in references])['score']
    rouge_result = rouge.compute(predictions=predictions, references=[r[0] for r in references], rouge_types=["rougeL"])


    return {
        "BLEU": bleu_,
        "chrF": chrf_,
        "ROUGE-L": rouge_result["rougeL"]}

In [ ]:
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

metrics_model_base = metrics(base_model, tokenizer, raw_test)

In [ ]:
model_ver1 = AutoModelForSeq2SeqLM.from_pretrained("my_model_ver1")
tokenizer_1 = AutoTokenizer.from_pretrained("my_model_ver1")

metrics_model_ver1 = metrics(model_ver1, tokenizer_1, raw_test)

In [ ]:
model_ver2 = AutoModelForSeq2SeqLM.from_pretrained("my_model_ver2")
tokenizer_2 = AutoTokenizer.from_pretrained("my_model_ver2")

metrics_model_ver2 = metrics(model_ver2, tokenizer_2, raw_test)

In [30]:
metrics_df = pd.concat([
    pd.DataFrame([metrics_model_base], index=["OPUS-MT"]),
    pd.DataFrame([metrics_model_ver1], index=["LoRA ver.1"]),
    pd.DataFrame([metrics_model_ver2], index=["LoRA ver.2"])
], axis=0)
metrics_df

,BLEU,chrF,ROUGE-L
OPUS-MT,0.195338,49.861544,0.502824
LoRA ver.1,0.303787,58.646219,0.611332
LoRA ver.2,0.508579,71.365081,0.722241


#### Как видно из метрик - LoRA сработал, неудивительно что модель второй версии показывает себя лучше, там дообучался полный attention и данных в целом было больше. 

#### Дальшет я попробую разобраться с gradio -> **UI_model**